In [1]:
!pip install -q transformers
!pip install -q torchdata

In [2]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
import torch
import torchtext
import torchdata
from transformers import BertModel, BertConfig
from torchtext.transforms import BERTTokenizer
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader

In [4]:
train_ds, test_ds = torchtext.datasets.IMDB(split=('train', 'test'))
train_data = DataLoader(train_ds, batch_size=1, shuffle=True)
test_data = DataLoader(test_ds, batch_size=1, shuffle=True)

In [5]:
import torch, torchtext
from torchtext.models import RobertaClassificationHead
from torchtext.functional import to_tensor
xlmr_large = torchtext.models.XLMR_LARGE_ENCODER
classifier_head = torchtext.models.RobertaClassificationHead(num_classes=2, input_dim = 1024)
model = xlmr_large.get_model(head=classifier_head).to(device)
transform = xlmr_large.transform()
# input_batch = list(b1)
# model_input = to_tensor(transform(input_batch), padding_value=1).to(device)
# output = model(model_input)
# output.shape

In [6]:
n_epochs = 1
n_classes = 2

In [27]:
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()
for epoch in range(n_epochs):
    for i, batch in enumerate(train_data):
        # if i == 500:
        #     break
        optim.zero_grad()
        # print(list(batch[1]))
        model_input = to_tensor(transform(list(batch[1])), padding_value=1).to(device)
        predict = model(model_input)
        loss = loss_func(predict.view(-1, n_classes),
                         batch[0].to(device).view(-1),)
        loss.backward()
        optim.step()
        torch.cuda.empty_cache()
        if i % 100 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')

epoch: 0, step: 0, loss: 0.0
epoch: 0, step: 100, loss: 0.0
epoch: 0, step: 200, loss: 0.0
epoch: 0, step: 300, loss: 0.0
epoch: 0, step: 400, loss: 0.0
epoch: 0, step: 500, loss: 0.0
epoch: 0, step: 600, loss: 0.0
epoch: 0, step: 700, loss: 0.0
epoch: 0, step: 800, loss: 0.0
epoch: 0, step: 900, loss: 0.0
epoch: 0, step: 1000, loss: 0.0
epoch: 0, step: 1100, loss: 0.0
epoch: 0, step: 1200, loss: 0.0
epoch: 0, step: 1300, loss: 0.0
epoch: 0, step: 1400, loss: 0.0
epoch: 0, step: 1500, loss: 0.0
epoch: 0, step: 1600, loss: 0.0
epoch: 0, step: 1700, loss: 0.0
epoch: 0, step: 1800, loss: 0.0
epoch: 0, step: 1900, loss: 0.0
epoch: 0, step: 2000, loss: 0.0
epoch: 0, step: 2100, loss: 0.0
epoch: 0, step: 2200, loss: 0.0
epoch: 0, step: 2300, loss: 0.0
epoch: 0, step: 2400, loss: 0.0


In [30]:
torch.save(model.state_dict(), f'./model_text_{epoch}.pth')

In [8]:
torch.cuda.empty_cache()

In [28]:
model.eval()
pred = []
tar = []
for i, batch in enumerate(test_data):
  # print(batch[0])
  if i == 500:
    break
  res = model(to_tensor(transform(list(batch[1])), padding_value=1).to(device))
  if res[0][0] > res[0][1]:
    res_class = 0
  else: res_class = 1
  pred.append(res_class)
  tar.append(int(batch[0][0]))
  # res_class = torch.tensor(res_class).unsqueeze(0)
  torch.cuda.empty_cache()
  # print(batch[0], res_class) 
# print(tar)
# print(pred)

In [29]:
from sklearn.metrics import accuracy_score

accuracy_score(tar, pred)

1.0